In [4]:
import qiskit

In [7]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.19.1', 'qiskit-aer': '0.10.2', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': None, 'qiskit': '0.34.1', 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [8]:
from qiskit import IBMQ

In [ ]:
IBMQ.save_account('fabd131bc4c5fcea32e3816f67eaf83b40b579312e461eb0ce6427a81e630c4cdb984e46c9a0639d09c7335b1cfef5aed7a07f2207f904dfee733a518be1fc70')

In [9]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>